In [ ]:
import os
import math
from data import testsets
import torch
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import numpy as np
from data.datasets import *
from trainers.distiller import Distiller
from torch.utils.data import DataLoader
from models.stmfnet import STMFNet
from models.student import student_STMFNet
import models
import losses
import datetime
from os.path import join
from torchinfo import summary
import pandas as pd

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
torch.cuda.get_device_name()

In [ ]:
# class testArgs:
#     gpu_id = 0
#     net = 'STMFNet'
#     dataset = 'Ucf101_quintuplet'
#     metrics = ['PSNR', 'SSIM']
#     checkpoint = './train_results/checkpoint/model_epoch008.pth'
#     # checkpoint = './models/stmfnet.pth'
#     data_dir = 'D:/stmfnet_data'
#     out_dir = './tests/results'
#     featc = [64, 128, 256, 512]
#     featnet = 'UMultiScaleResNext'
#     featnorm = 'batch'
#     kernel_size = 5
#     dilation = 1
#     finetune_pwc = False

# class trainArgs:
#     gpu_id = 0
#     net = 'STMFNet'
#     data_dir = 'D:/stmfnet_data'
#     out_dir = './train_results'
#     load = None
#     epochs = 70
#     batch_size = 2
#     loss = "1*Lap"
#     patch_size = 256
#     lr = 0.001
#     lr_decay = 20
#     decay_type = 'step'
#     gamma = 0.5
#     patience = None
#     optimizer = 'ADAMax'
#     weight_decay = 0
#     featc = [32, 64, 96, 128]
#     featnet = 'UMultiScaleResNext'
#     featnorm = 'batch'
#     kernel_size = 5
#     dilation = 1
#     finetune_pwc = False
#     teacher = "STMFNet"
#     student = "student_STMFNet"
#     temp = 10
#     alpha = 0.1
#     distill_loss_fn = "KLDivLoss"

# args=trainArgs()

### Weights Comparison Automation:

define args

In [ ]:
# class trainArgs:
#     gpu_id = 0
#     net = 'STMFNet'
#     data_dir = 'D:/stmfnet_data'
#     out_dir = './train_results'
#     load = None
#     epochs = 70
#     batch_size = 2
#     loss = "1*Lap"
#     patch_size = 256
#     lr = 0.001
#     lr_decay = 20
#     decay_type = 'step'
#     gamma = 0.5
#     patience = None
#     optimizer = 'ADAMax'
#     weight_decay = 0
#     featc = [32, 64, 96, 128]
#     featnet = 'UMultiScaleResNext'
#     featnorm = 'batch'
#     kernel_size = 5
#     dilation = 1
#     finetune_pwc = False
#     temp = 10
#     alpha = 0.3
#     distill_loss_fn = 'KLDivLoss'

# args=trainArgs()

load models:

In [ ]:
# def to_device(data, device):
#     if isinstance(data, (list,tuple)):
#         return [to_device(x, device) for x in data]
#     return data.to(device, non_blocking=True)

# teacher = to_device(STMFNet(args), device)
# teacher.to(device)
# t_checkpoint = torch.load('./models/stmfnet.pth')
# teacher.load_state_dict(t_checkpoint['state_dict'])


# model = to_device(STMFNet(args), device)
# model.to(device)
# m_checkpoint = torch.load("./train_results/checkpoint/model_epoch016.pth")
# model.load_state_dict(m_checkpoint['state_dict'])

In [ ]:
# student = to_device(student_STMFNet(args), device)
# student.to(device)
# m_checkpoint = torch.load("./distill_results/checkpoint/model_epoch001.pth")
# student.load_state_dict(m_checkpoint['state_dict'])

In [ ]:
# student_summary = summary(student, [(2, 3, 256, 256), (2, 3, 256, 256), (2, 3, 256, 256), (2, 3, 256, 256)])
# with open('./summaries/student.txt', 'w', encoding="utf-8") as f:
#     f.write(str(student_summary))
#     f.close()

calculate number of zeroes in each layer:

In [ ]:
# teacher_weights = [w for name,
#                    w in teacher.named_parameters() if "weight" in name]
# model_weights = [w for name, w in model.named_parameters() if "weight" in name]
# layer_names = [name for name, w in model.named_parameters()
#                if "weight" in name and len(w.shape) > 1]
# K_parameters = np.array([np.prod(w.shape)
#                         for w in model_weights if len(w.shape) > 1])
# layer_shapes = [w.shape for w in model_weights if len(w.shape) > 1]
# num_zeros = np.array([np.count_nonzero(w.detach().cpu() == 0)
#                      for w in model_weights if len(w.shape) > 1])

find compression ratios for each layer:

In [ ]:
# compression_ratios = 1 - (num_zeros / K_parameters)


# layer_shapes_df = pd.DataFrame(layer_shapes, columns=['C_out','C_in','q1','q2','q3']).fillna(0)
# # df1 = pd.DataFrame(layer_shapes, columns=['layer_shapes'])
# df = pd.DataFrame(num_zeros, columns=['num_zeros'])
# df['compression_ratios'] = compression_ratios
# df['layer_shapes'] = layer_shapes
# # df['parameters'] = K_parameters
# data = pd.concat([layer_shapes_df, df], axis=1)


# data['new_C_in'] = round(data['C_in'] * data['compression_ratios'])
# data['difference'] = (abs(data['C_in'] - data['new_C_in'])) #  / ((data['C_in'] + data['new_C_in'])/2)

# print(max(data['difference']))
# # data = data[data['difference'] > 0.3]
# data['layer_names'] = layer_names
# data.to_csv('./layer_shapes.csv')

# data


model summaries:

In [ ]:
# teacher_summary = summary(teacher, [(2, 3, 256, 256), (2, 3, 256, 256), (2, 3, 256, 256), (2, 3, 256, 256)])
# with open('./summaries/teacher.txt', 'w', encoding="utf-8") as f:
#     f.write(str(teacher_summary))
#     f.close()

# model_summary = summary(model, [(2, 3, 256, 256), (2, 3, 256, 256), (2, 3, 256, 256), (2, 3, 256, 256)])
# with open('./summaries/model.txt', 'w', encoding="utf-8") as f:
#     f.write(str(model_summary))
#     f.close()

### Knowledge Distillation:

In [ ]:
class testArgs:
    gpu_id = 0
    net = "STMFNet"
    dataset = "Snufilm_extreme_quintuplet"
    metrics = ["PSNR", "SSIM"]
    # checkpoint = './models/stmfnet.pth'
    data_dir = "D:/stmfnet_data"
    out_dir = "./tests/results"
    featc = [32, 64, 96, 128]
    featnet = "UMultiScaleResNext"
    featnorm = "batch"
    kernel_size = 5
    dilation = 1
    finetune_pwc = False


class trainArgs:
    gpu_id = 0
    net = "STMFNet"
    data_dir = "D:/stmfnet_data"
    out_dir = "./train_results"
    load = None
    epochs = 70
    batch_size = 2
    loss = "1*Lap"
    patch_size = 256
    lr = 0.001
    lr_decay = 20
    decay_type = "step"
    gamma = 0.5
    patience = None
    optimizer = "ADAMax"
    weight_decay = 0
    featc = [32, 64, 96, 128]
    featnet = "UMultiScaleResNext"
    featnorm = "batch"
    kernel_size = 5
    dilation = 1
    finetune_pwc = False
    teacher = "STMFNet"
    student = "student_STMFNet"
    temp = 10
    alpha = 0.1
    distill_loss_fn = "KLDivLoss"

    metrics = ["PSNR", "SSIM"]


args = trainArgs()


### import data

In [ ]:
torch.cuda.set_device(args.gpu_id)

# training sets
vimeo90k_train = Vimeo90k_quintuplet(
    join(args.data_dir, "vimeo_septuplet"),
    train=True,
    crop_sz=(args.patch_size, args.patch_size),
)
bvidvc_train = BVIDVC_quintuplet(
    join(args.data_dir, "bvidvc"), crop_sz=(args.patch_size, args.patch_size)
)

# validation set
vimeo90k_valid = Vimeo90k_quintuplet(
    join(args.data_dir, "vimeo_septuplet"),
    train=False,
    crop_sz=(args.patch_size, args.patch_size),
    augment_s=False,
    augment_t=False,
)

datasets_train = [bvidvc_train]
train_sampler = Sampler(datasets_train, iter=True)

# data loaders
train_loader = DataLoader(
    dataset=train_sampler, batch_size=args.batch_size, shuffle=True, num_workers=0
)
valid_loader = DataLoader(
    dataset=vimeo90k_valid, batch_size=args.batch_size, num_workers=0
)

### teacher model

In [ ]:
# Load the model

torch.cuda.set_device(args.gpu_id)

if not os.path.exists(args.out_dir):
    os.mkdir(args.out_dir)


teacher = getattr(models, args.teacher)(args).cuda()
def load_model(filepath):

    checkpoint = torch.load(filepath)
    model = STMFNet(args).cuda()
    model.load_state_dict(checkpoint['state_dict'])
    
    return model

model = load_model("./models/stmfnet.pth")

teacher = STMFNet(args).cuda()

### student model

In [ ]:
student = getattr(models, args.student)(args).cuda()
student.to(device);


### distillation model

In [ ]:
# args=trainArgs()

# softmax_optimiser = nn.Softmax(dim=1)
# mse_loss_function = nn.MSELoss()

# def my_loss(scores, targets, temperature = 5):
#     soft_pred = softmax_optimiser(scores / temperature)
#     soft_targets = softmax_optimiser(targets / temperature)
#     loss = mse_loss_function(soft_pred, soft_targets)
#     return loss

# distil_optimizer = optim.Adam(student.parameters(), lr=0.0001)

# losses = []

# for epoch in range(5):

# 	running_loss = 0.0
# 	for i, data in enumerate(train_loader, 1):

# 		inputs, labels = data[0].to(device), data[1].to(device)

# 		targets = teacher(inputs)
# 		scores = student(inputs)
# 		loss = my_loss(scores, targets, temperature = 2)
# 		distil_optimizer.zero_grad()
# 		loss.backward()
# 		distil_optimizer.step()

# 		# print statistics
# 		running_loss += loss.item()
# 		if i % 60 == 59:    # print every 60 mini-batches
# 			print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 60:.3f}')
# 			running_loss = 0.0
		
# 	print('appending loss: ', loss.item())
# 	losses.append(loss.item())

In [ ]:
# from torchinfo import summary


# student.parameters

In [ ]:
# args = trainArgs()
# args.loss = "1*Lap"

# import losses
# loss = losses.DistillationLoss(args)

# start_epoch = 0
# # if args.load is not None:
# #     checkpoint = torch.load(args.load)
# #     student.load_state_dict(checkpoint["state_dict"])
# #     start_epoch = checkpoint["epoch"]

# distill_optimizer = optim.Adam(student.parameters(), lr=0.0001)
# my_trainer = Distiller(args, train_loader, valid_loader, student, teacher, loss, start_epoch)

# # now = datetime.datetime.now().strftime("%Y-%m-%d-%H:%M:%S")
# # with open(join(args.out_dir, "config.txt"), "a") as f:
# #     f.write(now + "\n\n")
# #     for arg in vars(args):
# #         f.write("{}: {}\n".format(arg, getattr(args, arg)))
# #     f.write("\n")

# # while not my_trainer.terminate():
# #     my_trainer.train()
# #     my_trainer.save_checkpoint()
# #     my_trainer.validate()


In [ ]:
args=testArgs()
args.dataset = 'Ucf101_quintuplet'

def to_device(data, device):
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

def getmax():
    nums = []
    for file in os.listdir('./train_results/checkpoint'):
        if file.endswith('.pth'):
            nums.append(int(file.split('_epoch')[-1].split('.')[0]))
    return max(nums)

# for i in range(getmax()):
model = to_device(STMFNet(args), device)
model.to(device)
model_name = "model_epoch" + str(31).zfill(3)
model_path = "./train_results/checkpoint/" + model_name + ".pth"

model_path = "./models/stmfnet.pth"

checkpoint = torch.load(model_path)
model.load_state_dict(checkpoint['state_dict'])

print("Testing ", model_name, " on dataset: ", args.dataset)
test_dir = os.path.join(args.out_dir, args.dataset)
if args.dataset.split("_")[0] in ["VFITex", "Ucf101", "Davis90"]:
    db_folder = args.dataset.split("_")[0].lower()
else:
    db_folder = args.dataset.lower()
test_db = getattr(testsets, args.dataset)(os.path.join(args.data_dir, db_folder))
if not os.path.exists(test_dir):
    os.mkdir(test_dir)

test_db.eval(model, metrics=args.metrics, output_dir=test_dir)

In [ ]:
# Python program to show time by process_time() 
from time import process_time
  
# assigning n = 50 
n = 50
  
# Start the stopwatch / counter 
t1_start = process_time() 
   
for i in range(n):
    print(i, end =' ')
  
print() 
  
# Stop the stopwatch / counter
t1_stop = process_time()
   
print("Elapsed time:", t1_stop, t1_start) 
   
print("Elapsed time during the whole program in seconds:",
                                         t1_stop-t1_start) 

In [ ]:
metrics=["PSNR", "SSIM"]
results_dict = {k: [] for k in metrics}
results_dict["Times"] = []
results_dict["Frames"] = []

results_dict['Times'].append(16)
results_dict['Times'].append(14)

results_dict['Frames'].append(38)
results_dict['Frames'].append(32)



np.sum(results_dict["Times"]) / np.sum(results_dict["Frames"])

In [ ]:
# from torchvision import transforms
# from PIL import Image
# from os.path import join, exists
# import utility
# from torchvision.utils import save_image as imwrite

# db_dir = 'D:/stmfnet_data/ucf101'
# transform = transforms.Compose([transforms.ToTensor()])

# im_list = os.listdir(db_dir)

# input1_list = []
# input3_list = []
# input5_list = []
# input7_list = []
# gt_list = []
# for item in im_list:
#     input1_list.append(
#         transform(Image.open(join(db_dir, item, "frame0.png")))
#         .cuda()
#         .unsqueeze(0)
#     )
#     input3_list.append(
#         transform(Image.open(join(db_dir, item, "frame1.png")))
#         .cuda()
#         .unsqueeze(0)
#     )
#     input5_list.append(
#         transform(Image.open(join(db_dir, item, "frame2.png")))
#         .cuda()
#         .unsqueeze(0)
#     )
#     input7_list.append(
#         transform(Image.open(join(db_dir, item, "frame3.png")))
#         .cuda()
#         .unsqueeze(0)
#     )
#     gt_list.append(
#         transform(Image.open(join(db_dir, item, "framet.png")))
#         .cuda()
#         .unsqueeze(0)
#     )

# # def eval(model, , output_dir=None, output_name="output.png"):
# # model.eval()

# output_dir = "./tests/"
# output_name = "output.png"


# # results_dict = {k: [] for k in metrics}

# # logfile = open(join(output_dir, "results.txt"), "a")

# for idx in range(len(im_list)):
#     if not exists(join(output_dir, im_list[idx])):
#         os.makedirs(join(output_dir, im_list[idx]))

#     with torch.no_grad():
#         out = teacher(
#             input1_list[idx],
#             input3_list[idx],
#             input5_list[idx],
#             input7_list[idx],
#         )
#     gt = gt_list[idx]


#     imwrite(out, join(output_dir, im_list[idx], output_name), range=(0, 1))

# #     msg = (
# #         "{:<15s} -- {}".format(
# #             im_list[idx],
# #             {k: round(results_dict[k][-1], 3) for k in metrics},
# #         )
# #         + "\n"
# #     )
# #     print(msg, end="")
# #     logfile.write(msg)

# # msg = (
# #     "{:<15s} -- {}".format(
# #         "Average", {k: round(np.mean(results_dict[k]), 3) for k in metrics}
# #     )
# #     + "\n\n"
# # )
# # print(msg, end="")
# # logfile.write(msg)
# # logfile.close()
# torch.cuda.empty_cache()

Layer Shape Predictions:

In [ ]:
def getmax():
    nums = []
    for file in os.listdir('./train_results/checkpoint'):
        if file.endswith('.pth'):
            nums.append(int(file.split('_epoch')[-1].split('.')[0]))
    return max(nums)

df = pd.DataFrame(compression_ratios, columns=['compression_ratios'])
for i in range(getmax()):
    model = to_device(STMFNet(args), device)
    model.to(device)
    model_name = "model_epoch" + str(i+1).zfill(3)
    model_path = "./train_results/checkpoint/" + model_name + ".pth"
    checkpoint = torch.load(model_path)
    model.load_state_dict(checkpoint['state_dict'])

    teacher_weights = [w for name,
                    w in teacher.named_parameters() if "weight" in name]
    model_weights = [w for name, w in model.named_parameters() if "weight" in name]
    layer_names = [name for name, w in model.named_parameters()
                if "weight" in name and len(w.shape) > 1]
    K_parameters = np.array([np.prod(w.shape)
                            for w in model_weights if len(w.shape) > 1])
    layer_shapes = [w.shape for w in model_weights if len(w.shape) > 1]
    num_zeros = np.array([np.count_nonzero(w.detach().cpu() == 0)
                        for w in model_weights if len(w.shape) > 1])
    compression_ratios = 1 - (num_zeros / K_parameters)
 
    weights = [w for name, w in model.named_parameters() if "weight" in name]
    num_features = sum([w.numel() for w in weights])
    density = sum([torch.sum(w != 0).item() for w in weights]) / num_features
    df[str(density)] = compression_ratios

df.to_csv('./predictions.csv')